<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/Image_processing_planet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
install.packages('gdalutils')
install.packages('sf')
install.packages('raster')
install.packages('exactextractr')
install.packages("googledrive")
install.packages("googletoken")
install.packages("DBI")
install.packages("RSQLite")
install.packages("RPostgreSQL")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘gdalutils’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘googletoken’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installin

# New Section

In [5]:
library(ggplot2)
library(sf)
library(magrittr)
library(data.table)
library(raster)
library(exactextractr)
library(googledrive)
library(DBI)
library(zip)

In [7]:
drive_auth(use_oob = TRUE, cache = FALSE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=cc24847bf9d11fae24ae2cb01a41770d&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQWZKb2hYbUtBTzJOaGJJeFJodGl0ekVpdkFNQTJ6YVBFZk1DbnpwbzVUamtyVXF3WG1TZmIyUlhKZTl4bUZ5c1AyWHRyZyIsInN0YXRlIjoiY2MyNDg0N2JmOWQxMWZhZTI0YWUyY2IwMWE0MTc3MGQifQ==


In [180]:
dir.create('/content/kahuku2019/')

In [181]:
# kahuku
# CCAP:  https://drive.google.com/file/d//view?usp=drive_link
drive_download(file = as_id('1rmLeIkZKapy580evTocdFtRlSJZNqt3e'), path =  '/content/kahuku2018/kahuku2018_ccap.tif', overwrite = TRUE)

File downloaded:

• kahuku.tif <id: 1rmLeIkZKapy580evTocdFtRlSJZNqt3e>

Saved locally as:

• /content/kahuku2018/kahuku2018_ccap.tif



In [182]:
#  kawinuimarsh 2020
# Planet Labs:
drive_download(file = as_id('19VQJrQdjldBPnGLqEUlXlTYMaPbYEb3E'), path =  '/content/kahuku2019/01.tif', overwrite = TRUE)
drive_download(file = as_id('17q83cY0iQ9gryUD-TXvzebRKF0sFtF9t'), path =  '/content/kahuku2019/02.tif', overwrite = TRUE)
drive_download(file = as_id('17YdI4uccBryL3e9r4LKdbM2EQeKVDeKn'), path =  '/content/kahuku2019/03.tif', overwrite = TRUE)
drive_download(file = as_id('1AeA3u1liji68-OMQ75ysT29OpBYTLvAp'), path =  '/content/kahuku2019/04.tif', overwrite = TRUE)
drive_download(file = as_id('18WXqsWMCI6RaC8birSoSPwWjW5swUf-V'), path =  '/content/kahuku2019/05.tif', overwrite = TRUE)
drive_download(file = as_id('17sjWS8dhcD6lk6TmbMqh7DTw31d15XI3'), path =  '/content/kahuku2019/06.tif', overwrite = TRUE)
drive_download(file = as_id('1ANrHeSNEHiAtWDMD5erEr0MbJqnjXDVs'), path =  '/content/kahuku2019/07.tif', overwrite = TRUE)
drive_download(file = as_id('19MF-VyY4uLhJW_sR2qfGwJESgCaKp2Mi'), path =  '/content/kahuku2019/08.tif', overwrite = TRUE)
drive_download(file = as_id('18WpzxSP3BcyCUZLGAvy6hchKdC3Awe3Y'), path =  '/content/kahuku2019/09.tif', overwrite = TRUE)
# drive_download(file = as_id(''), path =  '/content/kahuku2019/10.tif', overwrite = TRUE)
# drive_download(file = as_id(''), path =  '/content/kahuku2019/11.tif', overwrite = TRUE)

# https://drive.google.com/open?id=&usp=drive_copy
# https://drive.google.com/open?id=&usp=drive_copy
# https://drive.google.com/open?id=&usp=drive_copy
# https://drive.google.com/open?id=&usp=drive_copy
# https://drive.google.com/open?id=&usp=drive_copy
# https://drive.google.com/open?id=&usp=drive_copy
# https://drive.google.com/open?id=&usp=drive_copy
# https://drive.google.com/open?id=&usp=drive_copy
# https://drive.google.com/open?id=&usp=drive_copy

File downloaded:

• 2019-01-20_strip_2041858_composite.tif
  <id: 19VQJrQdjldBPnGLqEUlXlTYMaPbYEb3E>

Saved locally as:

• /content/kahuku2019/01.tif

File downloaded:

• 2019-03-15_strip_2206055_composite.tif
  <id: 17q83cY0iQ9gryUD-TXvzebRKF0sFtF9t>

Saved locally as:

• /content/kahuku2019/02.tif

File downloaded:

• 2019-05-21_strip_2382113_composite.tif
  <id: 17YdI4uccBryL3e9r4LKdbM2EQeKVDeKn>

Saved locally as:

• /content/kahuku2019/03.tif

File downloaded:

• 2019-06-06_strip_2423787_composite.tif
  <id: 1AeA3u1liji68-OMQ75ysT29OpBYTLvAp>

Saved locally as:

• /content/kahuku2019/04.tif

File downloaded:

• 2019-06-06_strip_2425031_composite.tif
  <id: 18WXqsWMCI6RaC8birSoSPwWjW5swUf-V>

Saved locally as:

• /content/kahuku2019/05.tif

File downloaded:

• 2019-07-24_strip_2546409_composite.tif
  <id: 17sjWS8dhcD6lk6TmbMqh7DTw31d15XI3>

Saved locally as:

• /content/kahuku2019/06.tif

File downloaded:

• 2019-10-07_strip_2733400_composite.tif
  <id: 1ANrHeSNEHiAtWDMD5erEr0MbJqn

ERROR: Error in curl::curl_fetch_disk(url, x$path, handle = handle): Operation was aborted by an application callback


In [ ]:
planet <- stack ('/content/kahuku2019/01.tif')
# planet
planet %>% plot


redo the ones where I forgot to include 03

In [ ]:
planet_raster <-stack(
  '/content/kahuku2019/01.tif',
'/content/kahuku2019/02.tif',
'/content/kahuku2019/03.tif',
'/content/kahuku2019/04.tif',
'/content/kahuku2019/05.tif',
'/content/kahuku2019/06.tif',
'/content/kahuku2019/07.tif',
'/content/kahuku2019/08.tif',
'/content/kahuku2019/09.tif')

In [ ]:
ccap_raster <- stack('/content/kahuku2018/kahuku2018_ccap.tif')

In [ ]:
planet_raster_extended <- extend(planet_raster, projectExtent(ccap_raster, planet_raster), value=NA)

In [ ]:
ccap_raster
planet_raster

In [ ]:
planet_raster %>% plot
# planet_raster %>% plot

In [ ]:
ccap_raster %>% plot

In [ ]:
ncol(ccap_raster)

nrow(ccap_raster)


y_end <- c(seq(1,ccap_raster %>% nrow,512),nrow(ccap_raster))[-1]
x_end <- c(seq(1,ccap_raster %>% ncol,512),ncol(ccap_raster))[-1]

extract_grid <- expand.grid(x_end,y_end) %>% data.table

names(extract_grid) <- c('x_end','y_end')

extract_grid$x_org <- extract_grid$x_end - 512
extract_grid$y_org <- extract_grid$y_end - 512

extract_grid$id <- 1:nrow(extract_grid)

In [ ]:
# dir.create('/TrainingData/Niihau')
# Specify the path for the new folder/directory
new_folder_path <- "/content/kahuku2019/Images"
new_folder_path2 <- "/content/kahuku2019/Labels"
# Use dir.create to create the new folder
dir.create(new_folder_path, showWarnings = FALSE)

# Check if the folder was created successfully
if (file.exists(new_folder_path)) {
  cat("Folder created:", new_folder_path, "\n")
} else {
  cat("Failed to create folder:", new_folder_path, "\n")
}


# Use dir.create to create the new folder
dir.create(new_folder_path2, showWarnings = FALSE)
# Check if the folder was created successfully
if (file.exists(new_folder_path2)) {
  cat("Folder created:", new_folder_path2, "\n")
} else {
  cat("Failed to create folder:", new_folder_path2, "\n")
}

In [ ]:
result_dt <-
1:nrow(extract_grid) %>% lapply(function(x){

  nrow <- 0
  ncol <- 0
  i <- 0
  while(nrow !=512 | ncol != 512){
    i <- i + 1
    # Calculate the geographic coordinates for the cropping extent
    xmin <- xFromCol(ccap_raster, extract_grid[x,]$x_org) + rnorm(1)*.01*2.4
    xmax <- xFromCol(ccap_raster, extract_grid[x,]$x_end) + rnorm(1)*.01*2.4
    ymin <- yFromRow(ccap_raster, extract_grid[x,]$y_end) + rnorm(1)*.01*2.4 # Note row_max as raster origin is top-left
    ymax <- yFromRow(ccap_raster, extract_grid[x,]$y_org) + rnorm(1)*.01*2.4 # Note row_min as raster origin is top-left

    e <- extent(xmin, xmax, ymin, ymax)

    ccap_cropped_raster <- crop(ccap_raster, e)

    nrow <- nrow(ccap_cropped_raster)
    ncol <- ncol(ccap_cropped_raster)


    # print(dim(ccap_cropped_raster))
    if(i>1000){return('failed to converge')}
    }

  ls_cropped_raster <- crop(planet_raster_extended, projectExtent(ccap_cropped_raster,planet_raster))

  ls_resampled_raster <- projectRaster(ls_cropped_raster,ccap_cropped_raster)
  print(ls_resampled_raster)
  pixel_table <- (table(values(ccap_cropped_raster))/ncell(ccap_cropped_raster)) %>% data.table

  names(pixel_table) <- c('label','percent')

  out_dt <- data.table(extract_grid[x,],xmin = xmin,xmax =xmax,ymin= ymin,ymax, pixel_table)

  if(ls_resampled_raster %>% nrow ==512 & ls_resampled_raster %>% ncol ==512 & ccap_cropped_raster%>% nrow ==512 & ccap_cropped_raster%>% ncol ==512){
    ls_resampled_raster %>%
    writeRaster(., paste("/content/kahuku2019/Images/",extract_grid[x,]$x_org,'_',extract_grid[x,]$y_org,'.tif',sep=''),overwrite=T)

    ccap_cropped_raster %>%
    writeRaster(., paste("/content/kahuku2019/Labels/",extract_grid[x,]$x_org,'_',extract_grid[x,]$y_org,'.tif',sep=''),overwrite=T)
  }else(print('uh oh'))

  return(out_dt)
})
result_dt[ result_dt %>% lapply(function(x){is.data.table(x)}) %>% unlist] %>% rbindlist %>%
fwrite("/content/kahuku2019/kahuku2019.csv")


In [ ]:
zip(zipfile = "/content/kahuku2019.zip",
    files = c("/content/kahuku2019/Images",
              "/content/kahuku2019/Labels",
              "/content/kahuku2019/kahuku2019.csv"))

In [ ]:
# save to processed_data https://drive.google.com/drive/folders/1d_Ix2hIbwtT2XAqtzncui2jPq13vKOoe?usp=drive_link
drive_upload(media = "/content/kahuku2019.zip", path = as_id("1d_Ix2hIbwtT2XAqtzncui2jPq13vKOoe"))